In [0]:
### Run this cell ONLY for Colab users ###
!unzip data.zip
# Otherwise, put the /data folder (containing two subfolders of 100 .txt files)
# under the working directory.
# Now we should have /data under the working directory.

In [0]:
# Upload correction_lib.py and Create_Words_Dictionary.py to Colab,
# or make sure they are in the same directory of this file (for local machine).
# Now, import correction_lib and Create_Words_Dictionary modules. 
import correction_lib as corr
from word_dict import Create_Words_Dictionary, Create_Word_Pair

In [0]:
import string
import glob
import os
import itertools
import collections
import timeit
import random
import pandas as pd
import numpy as np

In [0]:
def clean_word(w):
    out = []
    for c in w:
        c = c.lower()
        # Searching set is faster than list: O(1) vs. O(n=26)
        if c in set(string.ascii_lowercase):
            out.append(c)
    return(''.join(out))
# clean_word('Caat13.#abE')

In [0]:
def char_to_index(c):
    return(ord(c) - ord('a'))

In [0]:
# print matrices/digrams in a clear manner 
def print_matrix(matrix):
    alphabet = ' ' + string.ascii_lowercase
    print('  '.join(alphabet))
    for i in range(len(matrix)):
        print(chr(ord('a')+i), matrix[i])
# print_digram(digrams_by_len[3][(0, 1)])

#Error Detection

In [0]:
# create a list of words from ground truth; include repeatition and order of words
word_list = []
gt_filenames = glob.glob(os.path.join(os.getcwd(), 'data', 'ground_truth', '*.txt'))

count = 0
for gt_f in gt_filenames:
    with open(gt_f) as file:
        raw = file.read()
        # Split file content into words (by '\n', '\t', ' ', etc.)
        uncleaned_words = raw.split()
        # Clean up words, leave only all-alpha chars of length > 1 (function programming)
        word_list += list(filter(lambda x: 1 < len(x) < 21, map(clean_word, uncleaned_words)))
# print(len(word_list))
# print(word_list[:20])
# word_set = set(word_list)
# print(len(word_set))
# print(list(word_set)[:20])

In [0]:
# Categorize ground truth words by their length
group_by_len = collections.defaultdict(list)
for w in word_set:
    group_by_len[len(w)].append(w)

# A dictionary of positional binary digrams (matrices),
# ordered by word length and then by binary positions
digrams_by_len = collections.defaultdict(dict)
for length in group_by_len:
    for i, j in itertools.combinations(range(length), 2):
        key = (i, j)
        matrix = [[0] * 26 for _ in range(26)]
        for w in group_by_len[length]:
            matrix[char_to_index(w[i])][char_to_index(w[j])] = 1
        digrams_by_len[length][key] = matrix
#     print(length, len(digrams_by_len[length].keys()))

In [0]:
# Create a list of words from tesseract text; regard repeatition and order of words
tr_word_list = []
tr_filenames = glob.glob(os.path.join(os.getcwd(), 'data', 'tesseract', '*.txt'))
for tr_f in tr_filenames:
    with open(tr_f) as file:
        raw = file.read()
        uncleaned_words = raw.split()
        tr_word_list += list(filter(lambda x: 1 < len(x) < 21, map(clean_word, uncleaned_words)))
N_tr = len(tr_word_list)
# print(N_tr, '\n', tr_word_list[:30])

In [0]:
# A list of 3-tuples, each consisting of (detected error, left word, right word)
detected_error_tuples = []
for idx, w in enumerate(tr_word_list):
    error = False
    for i, j in itertools.combinations(range(len(w)), 2):
#         print(i, j, w[i], w[j], len(digrams_by_len[len(w)][(i, j)]), len(digrams_by_len[len(w)][(i, j)][0]))
        if not digrams_by_len[len(w)][(i, j)][char_to_index(w[i])][char_to_index(w[j])]:
            error = True
    if error:
        left = tr_word_list[i-1] if i > 0 else ''
        right = tr_word_list[i+1] if i < len(tr_word_list)-1 else ''
        detected_error_tuples.append((w, left, right))

# A list of detected error words
detected_error_words = [x[0] for x in detected_error_tuples]
        
# print(len(detected_error_tuples), len(detected_error_words))        
# print(detected_error_tuples[:10])
# print(detected_error_words[:10])

#Error Correction

In [0]:
# Create confusion matrices
Confusion = corr.Create_Confusion_Matrix()
# print_matrix(Confusion['Deletion_Confusion'])
# print_matrix(Confusion['Insertion_Confusion'])
# print_matrix(Confusion['Substitution_Confusion'])
# print_matrix(Confusion['Reversal_Confusion'])

# Useful values from section 3 of paper C-4.
N = len(word_list)
V = len(word_set)
denominator = N + V/2

In [0]:
# Input a detected error word;
# return a list of (correction candidates, changed letters),
# ordered by types of correction (from the ground truth word set)
def get_correction_candidates(w):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    candidate_list = [[] for _ in range(4)]
    
    ### 4 kinds of correction candidates (see Table 2, C-4)
    # 0. Deletion
    for i in range(len(w) + 1):
        for c in alphabet:
            correction = w[:i] + c + w[i:]
            if correction in word_set:
                diff = corr.find_deletion_letters(correction, w) 
                candidate_list[0].append((correction, diff['pre_letter'], diff['delete_letter']))
            
    # 1. Insertion
    for i in range(len(w)):
        correction = w[:i] + w[i+1:]
        if correction in word_set:
            diff = corr.find_insertion_letters(correction, w) 
            candidate_list[1].append((correction, diff['pre_letter'], diff['insert_letter']))
    
    # 2. Substitution
    for i in range(len(w)):
        for c in alphabet:
            if c != w[i]:
                correction = w[:i] + c + w[i+1:]
                if correction in word_set:
                    diff = corr.find_sub_rev_letters(correction, w) 
                    if diff['tag'] == 'sub':
                        candidate_list[2].append((correction, diff['pre_letter'], diff['changed_letter']))
    
    # 3. Reversal
    for i in range(len(w) - 1):
        correction = w[:i] + w[i+1] + w[i] + w[i+2:]
        if correction in word_set:
            diff = corr.find_sub_rev_letters(correction, w) 
            if diff['tag'] == 'rev':
                candidate_list[3].append((correction, diff['pre_letter'], diff['changed_letter']))
    
    return(candidate_list)
# get_correction_candidates('en')

In [48]:
# This cell needs approximately 90-120 seconds to run on Colab.
# Create a dictionary of all candidates of each detected error
start = timeit.default_timer()

all_candidates = collections.defaultdict(dict)
for word in detected_error_words:
    all_candidates[word] = get_correction_candidates(word)
    
stop = timeit.default_timer()
print('Time:', stop - start, 'seconds')

# print(len(detected_error_words))
# print(len(all_candidates))
# print(get_correction_candidates('hlth'))

Time: 11.063699515000053 seconds


In [0]:
# A dictionary of frequecies of words in the ground truth
word_freqs = collections.defaultdict(int)
for word in word_list:
    word_freqs[word] += 1
# print(dict((k, v) for k, v in word_freqs.items() if v >= 2))

# Pr(c) of all possible corrections (all words from ground truth)
corr_probs = collections.defaultdict(float)
for word, freq in word_freqs.items():
    corr_probs[word] = (freq + 0.5)/denominator
# print(corr_probs)

In [0]:
# Compute Pr(c), estimated by ELE (expected LE)
def get_Pr_c(correction):
    return(corr_probs[correction])

In [0]:
# chars[x] and chars[xy]
chars_x = [0] * 26 + [N]
chars_xy = [[0] * 26 for _ in range(27)]
for word in word_list:
    for i, c in enumerate(word):
        chars_x[char_to_index(c)] += 1
        if i:
            chars_xy[26][char_to_index(c)] += 1
        else:
            chars_xy[char_to_index(word[i-1])][char_to_index(c)] += 1
for i in range(len(chars_xy)):
    for j in range(len(chars_xy[0])):
        if not chars_xy[i][j]:
            chars_xy[i][j] = 0.5

In [0]:
def get_Pr_tc(typo, correction, pre, cur, error_type):
    if error_type == 'del':
        return Confusion['Deletion_Confusion'][char_to_index(pre)][char_to_index(cur)] \
               / chars_xy[char_to_index(pre)][char_to_index(cur)]
    if error_type == 'ins':
        return Confusion['Insertion_Confusion'][char_to_index(pre)][char_to_index(cur)] \
               / chars_x[char_to_index(pre)]
    if error_type == 'sub':
        return Confusion['Substitution_Confusion'][char_to_index(pre)][char_to_index(cur)] \
               / chars_x[char_to_index(pre)]
    if error_type == 'rev':
        return Confusion['Reversal_Confusion'][char_to_index(pre)][char_to_index(cur)] \
               / chars_xy[char_to_index(pre)][char_to_index(cur)]
# print(get_Pr_tc('ncreased', 'increased', '{', 'i', 'del'))

In [0]:
# A dictionary, the key of which is the words in ground truth,
# the value to each key records the two neighbor words and their frequencies.
neighbor_dict = Create_Words_Dictionary()
# neighbor_dict['reliable']

In [0]:
def get_Pr_context_correction(cand, left, right, method):
    """
    word:   a correction candidate
    left:   the left neighbor of the error word in tesseract
    right:  the right neighbor of the error word in tesseract
    method: 'MLE', or 'ELE' where r = freq + 0.5
    return: ELE of Pr(l|c) * Pr(r|c)
    """
    # r_left = freq of left appearing, r_right = freq of right appearing
    r_left, r_right = 0, 0
    if left in neighbor_dict[cand]['left']:
        r_left = neighbor_dict[cand]['left'][left]
    if right in neighbor_dict[cand]['right']:
        r_right = neighbor_dict[cand]['right'][right]
        
    if method == 'MLE':
        return(r_left*r_right)
    else: # method == 'ELE'
        return((r_left + 0.5)*(r_right + 0.5))

In [0]:
result = Create_Word_Pair()
word_pairs = result[:-1]
num_corrections_found_tesseract, sum_letters = result[-1][0], result[-1][1]
# print(sum_letters)

In [56]:
# This cell needs approximately 90-120 seconds to run on Colab.
start = timeit.default_timer()
detected_error_tuples2 = []

for t in detected_error_tuples:
    for pair in word_pairs:
        if t[0] == pair[1]:
            detected_error_tuples2.append((t[0], pair[0], t[1], t[2]))
            break
stop = timeit.default_timer() 
print('Time:', stop - start, 'seconds')
# print(len(detected_error_tuples2), detected_error_tuples2[:20])

Time: 122.72246861899998 seconds


In [0]:
def get_final_Pr_correction(cand, typo, pre, cur, error_type, left, right, method):
    return get_Pr_c(cand) * get_Pr_tc(typo, cand, pre, cur, error_type) \
           * get_Pr_context_correction(cand, left, right, method)

err_type = ['del', 'ins', 'sub', 'rev']
# A dictionary of the form: typo:(ground_truth, MLE_correction, ELE_correction)
result = {}
for typo, truth, left, right in detected_error_tuples2:
    cand_poss_mle, cand_poss_ele = [], []
    for e in range(4):
        for cand, pre, cur in all_candidates[typo][e]:
            mle_p = get_final_Pr_correction(cand, typo, pre, cur, err_type[e], left, right, 'MLE')
            ele_p = get_final_Pr_correction(cand, typo, pre, cur, err_type[e], left, right, 'ELE')
            cand_poss_mle.append((cand, mle_p))
            cand_poss_ele.append((cand, ele_p))
    mle_best_cand = max(cand_poss_mle, key=lambda x: x[1])[0] if cand_poss_mle else ''
    ele_best_cand = max(cand_poss_ele, key=lambda x: x[1])[0] if cand_poss_ele else ''
    result[typo] = (truth, mle_best_cand, ele_best_cand)

# print(len(result))
# print(dict((k, v) for k, v in result.items() if 2 < len(k) < 5))
# print(dict((k, v) for k, v in result.items() if v[0] == v[1]))
# print(len(dict((k, v) for k, v in result.items() if v[0] == v[1])))

# Performance Measure

In [0]:
# Word-wise recalls and precisions
num_corrections_found_MLE = num_corrections_found_tesseract\
                            + len(dict((k, v) for k, v in result.items() if v[0] == v[1]))
num_corrections_found_ELE = num_corrections_found_tesseract\
                            + len(dict((k, v) for k, v in result.items() if v[0] == v[2]))

wordwise_precision_tesseract = num_corrections_found_tesseract / N
wordwise_precision_MLE = num_corrections_found_MLE / N
wordwise_precision_ELE = num_corrections_found_ELE / N
# print(wordwise_precision_tesseract, wordwise_precision_MLE, wordwise_precision_ELE)

wordwise_recall_tesseract = num_corrections_found_tesseract / N_tr
wordwise_recall_MLE = num_corrections_found_MLE / N_tr
wordwise_recall_ELE = num_corrections_found_ELE / N_tr
# print(wordwise_recall_tesseract, wordwise_recall_MLE, wordwise_recall_ELE)

In [0]:
# Character-wise recalls and precisions
def get_100tage_per_word_pair(w1, w2):
    max_len, min_len = max(len(w1), len(w2)), min(len(w1), len(w2))
    score = 0
    for i in range(min_len):
        if w1[i] == w2[i]:
            score +=1
    return(score / max_len)
# print(get_100tage_per_word_pair('cors', 'car'))
    
# len(result) is denominator
sum_tesseract, sum_mle, sum_ele = 0, 0, 0
for k, v in result.items():
    sum_tesseract += get_100tage_per_word_pair(v[0], k)
    sum_mle += get_100tage_per_word_pair(v[0], v[1])
    sum_ele += get_100tage_per_word_pair(v[0], v[2])

# Total numbers of letters in ground truth and in tesseract
sum_all_truth, sum_all_tesseract = 0, 0
for w in word_list:
    sum_all_truth += len(w)
for w in tr_word_list:
    sum_all_tesseract += len(w)

charwise_precision_tesseract = sum_letters[1] / sum_all_truth
charwise_precision_MLE = (sum_letters[1] + sum_mle) / sum_all_truth
charwise_precision_ELE = (sum_letters[1] + sum_ele) / sum_all_truth
# print(charwise_precision_tesseract, charwise_precision_MLE, charwise_precision_ELE)

charwise_recall_tesseract = sum_letters[1] / sum_all_tesseract
charwise_recall_MLE = (sum_letters[1] + sum_mle) / sum_all_tesseract
charwise_recall_ELE = (sum_letters[1] + sum_ele) / sum_all_tesseract
# print(charwise_recall_tesseract, charwise_recall_MLE, charwise_recall_ELE)    

In [60]:
idxs = pd.Index(["Word-wise recall", "Word-wise precision", \
              "Character-wise recall ", "Character-wise precision"])
cols = pd.Index(["Tesseract", "Tesseract with Post-Processing (MLE)",\
              "Tesseract with Post-Processing (ELE)"])
df = pd.DataFrame(data=np.array([\
     [wordwise_precision_tesseract, wordwise_precision_MLE, wordwise_precision_ELE],\
     [wordwise_recall_tesseract, wordwise_recall_MLE, wordwise_recall_ELE],\
     [charwise_precision_tesseract, charwise_precision_MLE, charwise_precision_ELE],\
     [charwise_recall_tesseract, charwise_recall_MLE, charwise_recall_ELE]]),\
     index=idxs, columns=cols)
display(df)

,Tesseract,Tesseract with Post-Processing (MLE),Tesseract with Post-Processing (ELE)
Word-wise recall,0.627530,0.642320,0.643173
Word-wise precision,0.653725,0.669132,0.670021
Character-wise recall,0.894004,0.896862,0.896937
Character-wise precision,0.931114,0.934091,0.934169
